In [1]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /kaggle/working


In [2]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

In [ ]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

In [4]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [5]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/kaggle/working/weights/sam_vit_h_4b8939.pth ; exist: True


In [6]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [7]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [8]:
mask_generator = SamAutomaticMaskGenerator(sam)

In [9]:
mask_predictor = SamPredictor(sam)

In [ ]:
import base64

def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [10]:
import cv2
import numpy as np
import supervision as sv


In [11]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Union, Optional
from dataclasses_json import dataclass_json
from supervision import Detections


@dataclass_json
@dataclass
class COCOCategory:
    id: int
    name: str
    supercategory: str


@dataclass_json
@dataclass
class COCOImage:
    id: int
    width: int
    height: int
    file_name: str
    license: int
    date_captured: str
    coco_url: Optional[str] = None
    flickr_url: Optional[str] = None


@dataclass_json
@dataclass
class COCOAnnotation:
    id: int
    image_id: int
    category_id: int
    segmentation: List[List[float]]
    area: float
    bbox: Tuple[float, float, float, float]
    iscrowd: int


@dataclass_json
@dataclass
class COCOLicense:
    id: int
    name: str
    url: str


@dataclass_json
@dataclass
class COCOJson:
    images: List[COCOImage]
    annotations: List[COCOAnnotation]
    categories: List[COCOCategory]
    licenses: List[COCOLicense]


def load_coco_json(json_file: str) -> COCOJson:
    import json

    with open(json_file, "r") as f:
        json_data = json.load(f)

    return COCOJson.from_dict(json_data)


class COCOJsonUtility:
    @staticmethod
    def get_annotations_by_image_id(coco_data: COCOJson, image_id: int) -> List[COCOAnnotation]:
        return [annotation for annotation in coco_data.annotations if annotation.image_id == image_id]

    @staticmethod
    def get_annotations_by_image_path(coco_data: COCOJson, image_path: str) -> Optional[List[COCOAnnotation]]:
        image = COCOJsonUtility.get_image_by_path(coco_data, image_path)
        if image:
            return COCOJsonUtility.get_annotations_by_image_id(coco_data, image.id)
        else:
            return None

    @staticmethod
    def get_image_by_path(coco_data: COCOJson, image_path: str) -> Optional[COCOImage]:
        for image in coco_data.images:
            if image.file_name == image_path:
                return image
        return None

    @staticmethod
    def annotations2detections(annotations: List[COCOAnnotation]) -> Detections:
        class_id, xyxy = [], []

        for annotation in annotations:
            x_min, y_min, width, height = annotation.bbox
            class_id.append(annotation.category_id)
            xyxy.append([
                x_min,
                y_min,
                x_min + width,
                y_min + height
            ])

        return Detections(
            xyxy=np.array(xyxy, dtype=int),
            class_id=np.array(class_id, dtype=int)
        )

In [12]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lqigRvMBwxOG5h9OGHlq")
project = rf.workspace("deepu-c49hl").project("smoke_detection_rev01")
dataset = project.version(1).download("coco")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to smoke_detection_rev01-1 in coco:: 100%|██████████| 305/305 [00:00<00:00, 7911.43it/s]


In [22]:
import os

DATA_SET_SUBDIRECTORY = "valid"
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"
IMAGES_DIRECTORY_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY)
ANNOTATIONS_FILE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, ANNOTATIONS_FILE_NAME)

In [23]:
coco_data = load_coco_json(json_file=ANNOTATIONS_FILE_PATH)

CLASSES = [
    category.name
    for category
    in coco_data.categories
    if category.supercategory != 'none'
]

IMAGES = [
    image.file_name
    for image
    in coco_data.images
]

In [20]:
CLASSES

['Smoke']

In [ ]:

import random
random.seed(10)

In [24]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
output_dir = '/kaggle/working/masksVa'
os.makedirs(output_dir, exist_ok=True)
for i in IMAGES:
    EXAMPLE_IMAGE_PATH = os.path.join(dataset.location, DATA_SET_SUBDIRECTORY, i)
    annotations = COCOJsonUtility.get_annotations_by_image_path(coco_data=coco_data, image_path=i) 
    if not annotations:
        print(f"No annotations found for image: {i}")
        continue
    ground_truth = COCOJsonUtility.annotations2detections(annotations=annotations)

    ground_truth.class_id = ground_truth.class_id - 1
    image_bgr = cv2.imread(EXAMPLE_IMAGE_PATH)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    mask_predictor.set_image(image_rgb)
    masks, scores, logits = mask_predictor.predict(box=ground_truth.xyxy[0], multimask_output=True)
    mask_to_save = (masks[2] > 0.5).astype(np.uint8) * 255 
    mask_filename = os.path.splitext(i)[0] + '_mask.png' 
    mask_filepath = os.path.join(output_dir, mask_filename)

    cv2.imwrite(mask_filepath, mask_to_save)
    print(f"Saved mask for {i} as {mask_filepath}")

Saved mask for img_177_jpg.rf.1f9fd04441f3a9ef69159c85801b6201.jpg as /kaggle/working/masksVa/img_177_jpg.rf.1f9fd04441f3a9ef69159c85801b6201_mask.png
Saved mask for img_118_jpg.rf.21a15c8d8ebbca811dd5168ad802e47b.jpg as /kaggle/working/masksVa/img_118_jpg.rf.21a15c8d8ebbca811dd5168ad802e47b_mask.png
Saved mask for img_117_jpg.rf.238bc65f4947ea3be302a8754811b1ae.jpg as /kaggle/working/masksVa/img_117_jpg.rf.238bc65f4947ea3be302a8754811b1ae_mask.png
Saved mask for img_132_jpg.rf.2442e52fb119b741e25b42054f8a1617.jpg as /kaggle/working/masksVa/img_132_jpg.rf.2442e52fb119b741e25b42054f8a1617_mask.png
Saved mask for img_137_jpg.rf.29e1959b11854ac24ddb0ea5bdbf4b1e.jpg as /kaggle/working/masksVa/img_137_jpg.rf.29e1959b11854ac24ddb0ea5bdbf4b1e_mask.png
Saved mask for img_113_jpg.rf.48fde9fd172729b1fd983564d7a89708.jpg as /kaggle/working/masksVa/img_113_jpg.rf.48fde9fd172729b1fd983564d7a89708_mask.png
Saved mask for img_126_jpg.rf.4c4112c40b1da35c2c3f12d1316997fe.jpg as /kaggle/working/masksVa/

In [28]:
!zip -r masksVA.zip /kaggle/working/masksVa

  adding: kaggle/working/masksVa/ (stored 0%)
  adding: kaggle/working/masksVa/img_117_jpg.rf.238bc65f4947ea3be302a8754811b1ae_mask.png (deflated 30%)
  adding: kaggle/working/masksVa/img_178_jpg.rf.b85a4699001224d2aaeac63c9992d16c_mask.png (deflated 21%)
  adding: kaggle/working/masksVa/img_164_jpg.rf.dca57966f03f90c7399be165f4f01d27_mask.png (deflated 32%)
  adding: kaggle/working/masksVa/img_118_jpg.rf.21a15c8d8ebbca811dd5168ad802e47b_mask.png (deflated 23%)
  adding: kaggle/working/masksVa/img_137_jpg.rf.29e1959b11854ac24ddb0ea5bdbf4b1e_mask.png (deflated 28%)
  adding: kaggle/working/masksVa/img_113_jpg.rf.48fde9fd172729b1fd983564d7a89708_mask.png (deflated 38%)
  adding: kaggle/working/masksVa/img_126_jpg.rf.4c4112c40b1da35c2c3f12d1316997fe_mask.png (deflated 50%)
  adding: kaggle/working/masksVa/img_15_jpg.rf.e0c0014957be30b5586cefeba3764c0b_mask.png (deflated 9%)
  adding: kaggle/working/masksVa/img_145_jpg.rf.a3a5a1f00ed7a007e774aebf08261577_mask.png (deflated 27%)
  adding: k